In [1]:
import tensorflow as tf

In [2]:
node1 = tf.constant(3.0,tf.float32)
node2 = tf.constant(4.0,tf.float32)
print(node1, node2)

(<tf.Tensor 'Const:0' shape=() dtype=float32>, <tf.Tensor 'Const_1:0' shape=() dtype=float32>)


In [3]:
sess = tf.Session()
print(sess.run([node1,node2]))

[3.0, 4.0]


In [4]:
node3 = tf.add(node1, node2)
print('node3 :', node3)
print(sess.run(node3))

('node3 :', <tf.Tensor 'Add:0' shape=() dtype=float32>)
7.0


In [5]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b #+ provides a shortcut for tf.add
print(sess.run(adder_node,{a : 3, b: 4}))
print(sess.run(adder_node,{a : [0,1], b: [3,4]}))

7.0
[ 3.  5.]


In [6]:
# can make it slightly more complex
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a : 3, b : 4}))
print(sess.run(add_and_triple, {a  : [0,1], b: [3,4]}))

21.0
[  9.  15.]


In [7]:
# Variables
W = tf.Variable([.3],tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
lin_reg = W*x + b

In [8]:
init = tf.global_variables_initializer()
sess.run(init)

In [9]:
print(sess.run(lin_reg, {x : [1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [10]:
y = tf.placeholder(tf.float32)
sq_deltas = tf.square(lin_reg - y)
loss = tf.reduce_sum(sq_deltas)
print(sess.run(loss,{x: [1,2,3,4], y: [0,-1,-2,-3]}))

23.66


In [11]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


In [19]:
# tf.train API
optimizer = tf.train.GradientDescentOptimizer(0.01)#takes input lr
train = optimizer.minimize(loss)
sess.run(init) #reset all variables to initial values
for i in range(1000):
    sess.run(train,{x : [1,2,3,4], y : [0,-1,-2,-3]})
print (sess.run([W,b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


In [22]:
# tf.contrib.learn : Claims to make life easier
import tensorflow as tf
# np used to load, preprocess and manipulate data
import numpy as np
# declare list of features
# here we have only one type of real-valued feature
# There are many other types of columns that could be more useful
features = [tf.contrib.layers.real_valued_column("x",dimension=1)]
# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)
x = np.array([1,2,3,4])
y = np.array([0,-1,-2,-3])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size = 4, num_epochs = 1000)
# We can invoke 1000 training steps by invoking the `fit` method and passing the
# training data set
estimator.fit(input_fn = input_fn, steps = 1000)
# Here we evaluate how well our model did. In a real example, we would want
# to use a separate validation and testing data set to avoid overfitting.
print(estimator.evaluate(input_fn=input_fn))
estimator.get_params()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f94a3737ed0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:S

{'params': {'feature_columns': [_RealValuedColumn(column_name='x', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)],
  'gradient_clip_norm': None,
  'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead at 0x7f94a3719ad0>,
  'joint_weights': False,
  'optimizer': None}}

In [36]:
print(estimator.get_variable_names())
# estimator.get_variable_value(estimator.get_variable_names()[0])
for name in estimator.get_variable_names():
    print (name, estimator.get_variable_value(name))

['global_step', 'linear/bias_weight', 'linear/bias_weight/ear/bias_weight/part_0/Ftrl', 'linear/bias_weight/ear/bias_weight/part_0/Ftrl_1', 'linear/x/weight', 'linear/x/weight/linear/x/weight/part_0/Ftrl', 'linear/x/weight/linear/x/weight/part_0/Ftrl_1']
('global_step', 1000)
('linear/bias_weight', array([ 0.99839437], dtype=float32))
('linear/bias_weight/ear/bias_weight/part_0/Ftrl', array([ 53.72570419], dtype=float32))
('linear/bias_weight/ear/bias_weight/part_0/Ftrl_1', array([-36.590065], dtype=float32))
('linear/x/weight', array([[-0.99942887]], dtype=float32))
('linear/x/weight/linear/x/weight/part_0/Ftrl', array([[ 496.82763672]], dtype=float32))
('linear/x/weight/linear/x/weight/part_0/Ftrl_1', array([[ 111.3844986]], dtype=float32))


In [37]:
import numpy as np
import tensorflow as tf
# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # ModelFnOps connects subgraphs we built to the
  # appropriate functionality.
  return tf.contrib.learn.ModelFnOps(
      mode=mode, predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.contrib.learn.Estimator(model_fn=model)
# define our data set
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x}, y, 4, num_epochs=1000)

# train
estimator.fit(input_fn=input_fn, steps=1000)
# evaluate our model
print(estimator.evaluate(input_fn=input_fn, steps=10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f94a3489910>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpb_zeu9/model.ckpt.
INFO:tensorflow:loss = 16.0395364519, step = 1
INFO:tensorflow:global_step/sec: 1201.89
INFO:tensorflow:loss = 0.275874978346, step = 101 (0.084 sec)
INFO:tensorflow:global_step/sec: 1257.12
INFO:tensorflow:loss = 0.00780757485609, step = 201 (0.080 sec)
INFO:tensorflow:gl

In [41]:
estimator.

'/tmp/tmpb_zeu9'